<a href="https://colab.research.google.com/github/juancuevas-ops/Desregulaci-n-de-la-Producci-n-de-Interleucina-12/blob/main/CRITICAL_SHIFT_SARSCORV2_BETAVIRUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
pollicio_glicoprotena_de_superficie_s_sarscorv2_betavirus_path = kagglehub.dataset_download('pollicio/glicoprotena-de-superficie-s-sarscorv2-betavirus')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install Bio

In [ ]:
import os
import sys
from collections import Counter
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis


In [ ]:
protein_faa_file_path = "/kaggle/input/glicoprotena-de-superficie-s-sarscorv2-betavirus/proteina_S.faa"

In [ ]:
# Aminoácidos estándar a considerar
STANDARD_AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"

In [ ]:

# ==============================================================================
# --- FUNCIÓN PRINCIPAL DE ANÁLISIS DE COMPOSICIÓN ---
# ==============================================================================

def analyze_protein_composition(faa_file):
    """
    Analiza las secuencias de proteínas en un archivo FAA (FASTA Amino Acid)
    y calcula el porcentaje de cada aminoácido estándar.

    Args:
        faa_file (str): Ruta al archivo FAA.

    Returns:
        dict: {protein_id: {'length': int, 'aa_percentage': {aa: float}}}
              Retorna None si el archivo no se encuentra o hay un error.
    """
    if not os.path.exists(faa_file):
        print(f"Error: Archivo FAA '{faa_file}' no encontrado.", file=sys.stderr)
        return None

    results = {}
    processed_proteins = 0
    print(f"Info: Procesando archivo de proteínas: {faa_file}")

    try:
        for record in SeqIO.parse(faa_file, "fasta"):
            processed_proteins += 1
            protein_id = record.id
            # Obtener secuencia como string y limpiar (eliminar '*' de parada si existe)
            sequence = str(record.seq).upper().rstrip('*')
            seq_len = len(sequence)

            if seq_len == 0:
                print(f"Warning: Secuencia '{protein_id}' está vacía. Saltando.", file=sys.stderr)
                results[protein_id] = {'length': 0, 'aa_percentage': {aa: 0.0 for aa in STANDARD_AMINO_ACIDS}}
                continue

            # Calcular porcentajes usando ProteinAnalysis (maneja errores por AAs no estándar)
            # Nota: ProteinAnalysis ignora AAs no estándar (X, B, Z, etc.) en el conteo total.
            # Podríamos querer contarlos o excluirlos explícitamente si son comunes.
            try:
                analysed_seq = ProteinAnalysis(sequence)
                aa_percentages = analysed_seq.get_amino_acids_percent() # Devuelve dict {AA: percentage*100}
                # Formatear a porcentaje (dividir por 100 implícito en get_amino_acids_percent)
                # Asegurarse de que todos los AAs estándar están presentes, incluso si % es 0
                formatted_percentages = {aa: aa_percentages.get(aa, 0.0) * 100.0 for aa in STANDARD_AMINO_ACIDS}


            except ValueError as e:
                 # Manejar posible error si la secuencia contiene caracteres inválidos no manejados por ProteinAnalysis
                 print(f"Warning: Error analizando secuencia '{protein_id}'. Calculando manualmente. Error: {e}", file=sys.stderr)
                 # Cálculo manual como fallback (ignora AAs no estándar)
                 aa_counts = Counter(c for c in sequence if c in STANDARD_AMINO_ACIDS)
                 valid_len = sum(aa_counts.values())
                 if valid_len > 0:
                      formatted_percentages = {aa: (aa_counts.get(aa, 0) / valid_len) * 100.0 for aa in STANDARD_AMINO_ACIDS}
                 else:
                      formatted_percentages = {aa: 0.0 for aa in STANDARD_AMINO_ACIDS}


            results[protein_id] = {
                'length': seq_len,
                'aa_percentage': formatted_percentages
            }

        print(f"Info: Análisis de composición completado para {processed_proteins} proteína(s).")
        return results

    except Exception as e:
        print(f"Ocurrió un error inesperado durante el procesamiento del archivo FAA: {e}", file=sys.stderr)
        # import traceback; traceback.print_exc() # Para debug
        return None

# ==============================================================================
# --- EJECUCIÓN PRINCIPAL Y SALIDA ---
# ==============================================================================

if __name__ == "__main__":
    # Usar parámetros definidos en la sección de CONFIGURACIÓN
    protein_input_file = protein_faa_file_path

    # Ejecutar el análisis
    composition_results = analyze_protein_composition(protein_input_file)

    # Imprimir resultados
    if composition_results:
        print(f"\n===== COMPOSICIÓN DE AMINOÁCIDOS PARA: {protein_input_file} =====")
        print("\nNOTA: No hay 'valores de referencia' universales. La composición varía por proteína.")
        print("      Interpreta estos porcentajes en el contexto de cada proteína específica.\n")

        for protein_id, data in composition_results.items():
            print(f"--- Proteína: {protein_id} (Longitud: {data['length']} aa) ---")
            percentages = data['aa_percentage']
            # Imprimir porcentajes ordenados alfabéticamente
            sorted_aas = sorted(percentages.keys())
            output_lines = []
            for i in range(0, len(sorted_aas), 5): # Imprimir en grupos de 5 para legibilidad
                line = []
                for aa in sorted_aas[i:i+5]:
                    line.append(f"{aa}: {percentages[aa]:.2f}%")
                output_lines.append("   ".join(line))
            print("\n".join(output_lines))
            print("-" * 70)

    else:
        print("\nNo se generaron resultados. Verifica la ruta del archivo FAA y posibles errores.")

# Análisis de la glicoproteína de superficie S del SARS-CoV-2 y su relación con problemas cardíacos, renales, pulmonares y psiquiátricos
La glicoproteína de superficie S (proteína de espiga) del SARS-CoV-2 tiene una estructura compleja que permite la unión al receptor ACE2 en las células humanas. Los aminoácidos alterados en su secuencia (elevados o críticos) pueden influir en su función y, en consecuencia, en la patogénesis viral. A continuación, se relacionan los aminoácidos alterados con los efectos en los tres órganos mencionados y con trastornos psiquiátricos:

# Aminoácidos alterados y su impacto en el corazón
## Leucina (L, 8.48%)
Elevado
Arritmias y fibrosis miocárdica
La leucina es esencial para la estabilidad de proteínas estructurales. Un exceso puede favorecer la acumulación de colágeno anormal en el músculo cardíaco.

## Valina (V, 7.62%)
Elevado
Hipertensión y estrés oxidativo
La valina está implicada en la producción de radicales libres, lo que puede dañar el endotelio vascular y aumentar la presión arterial.

## Treonina (T, 7.62%)
Elevado
Disfunción endotelial
Modifica la señalización de proteínas vasculares, afectando la regulación de la presión arterial.

## Triptófano (W, 0.94%)
Crítico
Deficiencia de serotonina
El triptófano es precursor de serotonina, cuya deficiencia puede causar arritm

# Aminoácidos alterados y su impacto en los riñones
## Glutamato (E, 3.77%)
Elevado
Inflamación renal y daño tubular
El exceso de glutamato activa vías proinflamatorias (ej., NF-κB), dañando los túbulos renales.
## Fenilalanina (F, 6.05%)
Elevado
Acumulación de ácido úrico
La fenilalanina es precursora de metabolitos que generan ácido úrico, empeorando la función renal.

## Histidina (H, 1.34%)
Crítico
Deficiencia de histamina
La histidina es esencial para la producción de histamina, clave en la regulación de la presión arterial y la respuesta inmune.

## Serina (S, 7.78%)
Elevado
Daño glomerular
La serina participa en la síntesis de mucinas y proteínas extracelulares; su e

# Aminoácidos alterados y su impacto en los pulmones
## Leucina (L, 8.48%)
Elevado
Fibrosis pulmonar
Favorece la acumulación de colágeno tipo I, típica en la fibrosis.

## Serina (S, 7.78%)
Elevado
Inflamación alveolar
Altera la síntesis de surfactantes pulmonares (SP-B/SP-C), reduciendo la tensión superficial y causando distorsión visual.

## Treonina (T, 7.62%)
Elevado
Disfunción epitelial
Modifica la estructura de proteínas epiteliales, debilitando la barrera alveolar.

## Glicina (G, 6.44%)
Elevado
Matriz extracelular rígida
La glicina es crítica para el colágeno; su exceso genera rigidez tisular.

# Aminoácidos alterados y su relación con trastornos psiquiátricos
## Triptófano (W, 0.94%)
Crítico
Depresión y ansiedad
Deficiencia de triptófano reduce la síntesis de serotonina, un neurotransmisor clave para el estado de ánimo.

## Histidina (H, 1.34%)
Crítico
Síndrome de fatiga post-COVID
La histidina es precursora de histamina, reguladora del sueño y la alerta. Su déficit puede causar somnolencia y malestar cognitivo.

## Fenilalanina (F, 6.05%)
Elevado
Ansiedad y trastorno de ansiedad
El exceso de fenilalanina inhibe la absorción de triptófano al cerebro, desbalanceando la producción de serotonina.

## Alanina (A, 6.21%)
Elevado
Trastornos del estado de ánimo
La alanina elevada altera la homeostasis energética, afectando la produc

![](http://)

## Relación con la estructura de la proteína SARS-CoV-2
# Leucina (L, 8.48%) :
Región funcional : Presente en dominios de unión al receptor ACE2.
Impacto : Mutaciones en leucina (ej., L452R en variantes como Delta) aumentan la afinidad viral por ACE2, facilitando la entrada celular.

# Triptófano (W, 0.94%) :
Región funcional : Implicado en la conformación del dominio S1/S2 (zona de corte por furina).
Impacto : Deficiencia de triptófano en la proteína viral puede reducir su estabilidad, pero en humanos, su déficit afecta la reparación tisular.

# Histidina (H, 1.34%) :
Región funcional : Participa en la unión a iones metálicos y en la conformación de la proteína.
Impacto : Baja histidina en la proteína viral puede alterar su actividad, pero en humanos, reduce la producción de enzimas antioxidantes (ej., SOD2).

## Conexión con la patogénesis sistémica
Hipoglucemia reactiva :
La proteína SARS-CoV-2 induce la liberación de insulina mediante la activación de vías inflamatorias (ej., IL-6), lo que puede causar caídas de glucosa postprandial.
Estrés oxidativo :
Aminoácidos como fenilalanina (F) y glutamato (E) generan radicales libres, dañando células en corazón, riñón y pulmón.
Inflamación sistémica :
La proteína SARS-CoV-2 activa macrófagos y neutrófilos, liberando citoquinas (ej., TNF-α, IL-1β), lo que exacerba daños en los tres órganos.
## Recomendaciones clínicas
Monitoreo de aminoácidos :
Verificar niveles de triptófano, histidina y fenilalanina en pacientes con síntomas persistentes.
Antioxidantes :
N-acetilcisteína (NAC) : Reduce el estrés oxidativo en corazón, pulmón y riñón.
Moduladores de colágeno :
Pirfenidona/Nintedanib : Inhiben la fibrosis en órganos afectados.
Suplementos de triptófano :
Mejoran la producción de serotonina y reducen síntomas psiquiátricos.
Conclusión
Los aminoacidos alterados en la proteına SARS-CoV-2 (L, F, W, H) contribuyen a daño cardıaco, renal, pulmonar y psiquiatrico mediante inflamacion, estres oxidativo y desequilibrios metabolicos.
Analisis Bioinformatico Por Alejandro Cuevas VAsquez
Algoritmo Patentado por Ley 17336 PROPIEDAD INTELECTUAL
MINISTERIO DE EDUCACIÓN PÚBLICA CHILE.